In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3130,0496-AHOOK,Male,0,Yes,No,24,Yes,No,DSL,Yes,...,No,Yes,Yes,Yes,One year,No,Bank transfer (automatic),74.40,1712.9,No
7007,2274-XUATA,Male,1,Yes,No,72,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),63.10,4685.55,No
3518,8393-JMVMB,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.45,19.45,No
74,5630-AHZIL,Female,0,No,Yes,3,Yes,No,DSL,Yes,...,No,Yes,No,Yes,Month-to-month,Yes,Bank transfer (automatic),64.50,177.4,No
2725,4236-UJPWO,Female,0,No,No,2,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,24.50,46.4,No


In [ ]:
df.drop('customerID',axis='columns',inplace = True)
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object